# Análisis y visualización de datos mediante Folium y GeoPandas

[GeoPandas](http://geopandas.org/) es un proyecto de software libre que extiende los tipos de datos de [Pandas](http://pandas.pydata.org/) para incorporar objetos geométricos (puntos, líneas, polígonos, etc). GeoPandas se apoya en otras bibliotecas como [Fiona](https://github.com/Toblerity/Fiona) para el acceso a los datos (ej. en archivos), [Shapely](https://pypi.org/project/Shapely/) para geoprocesamiento y [Matplotlib](https://matplotlib.org/) para graficación.

Las dos estructuras principales de GeoPandas son:

- [GeoSeries](http://geopandas.org/data_structures.html#geoseries): es un vector en el que cada elemento es una geometría correspondiente a una observación. Por ejemplo, los polígonos de una capa de países.
- [GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe): es una estructura tabular (i.e. con filas y columnas) de datos geométricos y no geométricos (ej. textos, números). El conjunto de geometrías se implementa a través de GeoSeries. Un GeoDataFrame puede describirse como un DataFrame de Pandas con una columna adicional para geometrías.

**Instalación**

Para instalar GeoPandas mediante **conda**, debe ejecutarse la siguiente instrucción desde la línea de comandos de Anaconda (Anaconda Prompt):

```
conda install geopandas
```

**Importación de bibliotecas**

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

%matplotlib inline

import folium

**Variables globales**

In [ ]:
# Cantidad máxima de registros que se despliegan en un GeoDataFrame
pd.options.display.max_rows = 10

## Preparación de datos

**Carga de datos**

Para los siguientes ejemplos, se utilizará la capa de provincias y estados de [Natural Earth](https://www.naturalearthdata.com/), disponible en [https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-1-states-provinces/](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-1-states-provinces/) como un _shapefile_ y que fue convertida al formato GeoJSON. El archivo GeoJSON debe colocarse en el directorio de datos (/datos). Una vez hecho esto, se procede a cargar los datos en un GeoDataFrame, a través de la función [read_file()](http://geopandas.org/io.html).

In [ ]:
# Carga del archivo GeoJSON en un GeoDataFrame
provincias_gdf = gpd.read_file("datos/ne_10m_admin_1_states_provinces.geojson")

In [ ]:
# Verificación del tipo de datos de provincias_gdf
type(provincias_gdf)

In [ ]:
# Visualización de los primeros registros del GeoDataFrame
provincias_gdf.head()

In [ ]:
# Despliegue de las geometrías
provincias_gdf.geometry

In [ ]:
# provincias_gdf.geometry es una variable del tipo GeoSeries
type(provincias_gdf.geometry)

In [ ]:
# Información sobre provincias_gdf
provincias_gdf.info()

**Filtrado**

In [ ]:
# Por país
provincias_cr_gdf = provincias_gdf[provincias_gdf['admin'] == 'Costa Rica']

# Selección de un subconjunto de columnas
provincias_cr_gdf = provincias_cr_gdf[['admin','gns_name','geometry']]

provincias_cr_gdf

## Análisis de la información

### Mapeo

#### Matplotlib

La función [plot()](http://geopandas.org/reference.html#geopandas.GeoDataFrame.plot) proporciona una manera sencilla de visualizar los datos en un mapa.

In [ ]:
provincias_cr_gdf.plot(figsize=(15, 10))

La función [to_file()](http://geopandas.org/reference.html#geopandas.GeoDataFrame.to_file) guarda un GeoDataFrame en un formato geoespacial.

In [ ]:
# Se guarda el GeoDataFrame en un archivo
provincias_cr_gdf.to_file("datos/provincias_cr.geojson", driver="GeoJSON")

#### Folium

In [ ]:
# Mapa centrado en Costa Rica
m = folium.Map(location=[9.63, -84], zoom_start=8)

m

##### Capas GeoJSON

La clase [GeoJson](https://python-visualization.github.io/folium/modules.html#folium.features.GeoJson) crea una capa a partir de un archivo GeoJSON (o de un GeoDataFrame).

In [ ]:
# Se añade la capa de provincias como un objeto GeoJSON
folium.GeoJson("datos/provincias_cr.geojson", name='Provincias').add_to(m)

folium.LayerControl().add_to(m)

m

Seguidamente, se cargan los registros de presencia de _Ara macao_ en un GeoDataFrame, el cual se guarda como un archivo y luego se añade como una capa al mapa mediante la clase GeoJson de Folium.

**Capa de Ara macao**

In [ ]:
# Carga de datos en un DataFrame
ara_macao_df = pd.read_csv("datos/Ara-macao-CR-2018.csv", sep='\t')

# Reducción de columnas
ara_macao_df = ara_macao_df[['species', 'decimalLongitude', 'decimalLatitude', 'eventDate']]

ara_macao_df

In [ ]:
# Creación de geometrías para un GeoSeries
ara_macao_geom = [Point(xy) for xy in zip(ara_macao_df["decimalLongitude"], ara_macao_df["decimalLatitude"])]

# Creación de un GeoDataFrame
ara_macao_gdf=gpd.GeoDataFrame(ara_macao_df, crs={"init": "epsg:4326"}, geometry=ara_macao_geom)

ara_macao_gdf

In [ ]:
# Se guarda el GeoDataFrame en un archivo
ara_macao_gdf.to_file("datos/ara_macao.geojson", driver="GeoJSON")

Se construye nuevamente el mapa, con todas las capas

In [ ]:
# Mapa centrado en Costa Rica
m = folium.Map(location=[9.63, -84], zoom_start=8)

# Se añade la capa de provincias como un objeto GeoJSON
folium.GeoJson("datos/provincias_cr.geojson", name='Provincias').add_to(m)

# Se añade la capa de Ara macao como un objeto GeoJSON
folium.GeoJson("datos/ara_macao.geojson", name='Ara macao').add_to(m)

folium.LayerControl().add_to(m)

m

<div class="alert alert-success">
 <strong>Ejercicio</strong>:

Descargue del SNIT las capas de:
* Áreas de conservación [(http://www.snitcr.go.cr/servicios_ogc_lista_capas?k=bm9kbzo6NDA=&nombre=SINAC)](http://www.snitcr.go.cr/servicios_ogc_lista_capas?k=bm9kbzo6NDA=&nombre=SINAC)
* Red vial [(http://www.snitcr.go.cr/servicios_ogc_lista_capas?k=bm9kbzo6MjY=&nombre=IGN%20Cartograf%C3%ADa%201:5mil)](http://www.snitcr.go.cr/servicios_ogc_lista_capas?k=bm9kbzo6MjY=&nombre=IGN%20Cartograf%C3%ADa%201:5mil)

Agregue estas capas al mapa que se está construyendo en este notebook.

</div>